# OTI MONITORING SET UP - STEP 3 - UNIONING DATA & CREATING ALERTS
RUN IN TARGET ACCOUNT


## UNIONIZING DATA
1. Create a procedure that builds unioned views across all monitoring databases created in step 2 of the setup
2. Create a task that executes the view on a regular schedule 
3. Execute task
4. Verify views have been created
5. Vefify view contains unioned data

## CREATING ALERTS
1. Create alerting framework prerequisites
2. Create alerts to log events
3. Create alerts to email notifications
4. Designate users to receive alerts
5. Verify and resume alerts


In [ ]:
USE ROLE ACCOUNTADMIN;
grant create integration on account to role ACCOUNT_MONITOR_OTI;


USE ROLE ACCOUNT_MONITOR_OTI;
SET WH_NAME = 'TASTY_BI_WH';  --Warehouse used by the task that creates the unionized views


CREATE database IF NOT EXISTS ORG_MONITORING_UTILITIES; 
CREATE SCHEMA IF NOT EXISTS CENTRALIZED_DATA; 

USE database ORG_MONITORING_UTILITIES; 
USE SCHEMA  CENTRALIZED_DATA; 

### Create a procedure that builds unioned views across all monitoring databases created in step 2 of the setup


In [ ]:
CREATE OR REPLACE PROCEDURE UNION_BUILDER(DB_PREFIX VARCHAR)
RETURNS ARRAY
EXECUTE AS CALLER AS
$$
declare 
--create table cursor for roles
table_cursor CURSOR FOR
        select  object_name from object_list;
object_name varchar; 
msg_array array := array_construct();
error_array array:= array_construct();
begin
    show databases  in account;
    
    let dblist_stmt VARCHAR(16777216) :=
    ('create or replace temporary table account_shares as
    (select * from table(result_scan(last_query_id()))
    where "name" ilike '''||:db_prefix || '%'' 
    --and "kind" = ''IMPORTED DATABASE''
    )');
    
    execute immediate (:dblist_stmt) ;
    
    let db varchar :=  (select "name" from account_shares limit 1);
    
    show tables in database identifier(:db);
    create or replace temporary table object_list as 
    select "schema_name"||'.'||"name" as object_name
    from table(result_scan(last_query_id()))
    WHERE "schema_name" <> 'INFORMATION_SCHEMA';
    
    show views in database identifier(:db);
    
    insert into object_list 
    select "schema_name"||'.'||"name" as object_name
    from table(result_scan(last_query_id()))
    WHERE "schema_name" <> 'INFORMATION_SCHEMA';
    
    for var in table_cursor DO
        BEGIN
            object_name := var.object_name;
            let view_name varchar := split_part(var.object_name,'.',2);
            
            let ddl_stmt VARCHAR(16777216) := 
            (select 'CREATE OR REPLACE VIEW '||:view_NAME  ||' COPY GRANTS  AS '||'\n' ||rtrim(listagg('SELECT '''||"name"|| ''' AS ACCOUNT_DATABASE,
            * FROM '||"name"|| '.' ||:OBJECT_NAME  || ' union all '||'\n'),'union all '||'\n')||';'  from  account_shares);
            
            execute immediate (:ddl_stmt) ;
            
            EXCEPTION  -- log errors
                                 WHEN statement_error THEN
                                  LET errm := SQLERRM;
                                  error_array := array_append(error_array, 'ERROR creating view for ' ||:OBJECT_NAME   ||'.  Error Message: '||  :errm );
                           
                                 WHEN expression_error THEN
                                  LET errm := SQLERRM;
                                  error_array := array_append(error_array, 'ERROR creating view for ' ||:OBJECT_NAME   ||'.  Error Message: '||  :errm );
                           
                           
                                  WHEN other THEN
                                  LET errm := SQLERRM;
                                  error_array := array_append(error_array, 'ERROR creating view for ' ||:OBJECT_NAME   ||'.  Error Message: '||  :errm );
        END;
    END FOR;
    
    return (ERROR_array );

end ;

$$;;;


ALTER PROCEDURE UNION_BUILDER(varchar)
SET COMMENT = 
$$PROCEDURE NAME: UNION_BUILDER() 
PROCEDURE ARGUMENTS: DB_PREFIX  the prefix of the databases of inbound shares that contain account monitoring data 
PROCEDURE PURPOSE: To make a unioned views of tables in the monitoring databases that have been shared or replicated to the target account. 
PROCEDURE DETAILS:  The create a unionized view of each shared table so support alerting and reporting across all accounts at once;
$$;

### Create a task that executes the view on a regular schedule 


Current schedule is once a week.  Only need to run when objects are added to the data collection.

In [ ]:
--create task to execute procedures on a regular schedule 
--set to run every 24 hours at midnight CT but this can be configured 
CREATE OR REPLACE TASK TASK_CREATE_UNION_VIEWS
  SCHEDULE = 'USING CRON 0 4 * * 1 America/Chicago'  --once a week on mondays
  WAREHOUSE = $WH_NAME  --CHANGE WAREHOUSE 
AS
BEGIN
CALL UNION_BUILDER('MONITORING_SHARE');
END;

ALTER TASK TASK_CREATE_UNION_VIEWS RESUME ;


### Execute the task


In [ ]:
EXECUTE task TASK_CREATE_UNION_VIEWS;

### Monitor Task Execution

In [ ]:
--validate task status
SELECT * 
FROM TABLE(information_schema.task_history())
WHERE database_name = 'ORG_MONITORING_UTILITIES'
AND NAME = 'TASK_CREATE_UNION_VIEWS'
ORDER BY scheduled_time DESC;

###  Verify views have been created

In [ ]:
SHOW VIEWS IN SCHEMA;

### Vefify view contains unioned data

In [ ]:
SELECT * FROM  CENTRALIZED_DATA.TRUST_CENTER_SCANNER_AT_RISK_COUNT;

# CREATE ALERTING FRAMEWORK

## Alerting Prerequisites

- Notification Integration
- alert_event_log table
- event_recipients table
- alert_email_log table

In [ ]:
--Create Notification Integration 
CREATE NOTIFICATION INTEGRATION IF NOT EXISTS GENERAL_EMAIL_INT 
TYPE=EMAIL 
ENABLED=TRUE ; 

--event log table  --for logging captured events ()
CREATE TABLE  IF NOT EXISTS  alert_event_log
(
    event_id NUMBER AUTOINCREMENT START 1 INCREMENT 1,
    event_timestamp TIMESTAMP_NTZ,
    event_database varchar,
    event_type STRING,
    event_message STRING
);

--table to map event types to recipeints
CREATE TABLE  IF NOT EXISTS  event_recipients
(
    event_type STRING,
    notification_integration STRING,
    recipients_email STRING,
    frequency string
    
);

--create table for logging sent emails
create TABLE  IF NOT EXISTS  ALERT_EMAIL_LOG (
    alert_email_id VARCHAR(60),
	EMAIL_SUBJECT VARCHAR(16777216),
	NOTIFICATION_INTEGRATION VARCHAR(16777216),
	RECIPIENTS_EMAIL VARCHAR(16777216),
	EMAIL_EVENTS ARRAY,
    query_id varchar(60),
    error_code  int,
    error_message varchar(16777216),
    sent_time   timestamp default current_timestamp()
);

## Create Alerts that Log Events

- TC_CRITICAL_FINDINGS_ALERT
- ELEVATED_PRIVILEGES_ALERT
- ACCOUNT_PARAMETERS_ALERT


In [ ]:
--alert FOR CRITICAL TRUST CENTER FINDINGS
CREATE OR REPLACE ALERT TC_CRITICAL_FINDINGS_ALERT
  WAREHOUSE = $WH_NAME
  SCHEDULE = '1440 MINUTE'   
  IF (EXISTS (
        SELECT ACCOUNT_DATABASE,
                SEVERITY, SCANNER_ID,
                SCANNER_SHORT_DESCRIPTION, 
                CREATED_ON::DATE AS FINDING_DATE, 
                F.VALUE:entity_object_type::VARCHAR AS ENTITY_OBJECT_TYPE,
                F.VALUE:entity_id::VARCHAR AS ENTITY_ID,
                F.VALUE:entity_name::VARCHAR AS ENTITY_NAME,
                IMPACT, 
                SUGGESTED_ACTION,
                AT_RISK_ENTITIES,
                RANK() OVER (PARTITION BY ACCOUNT_DATABASE,SCANNER_ID ORDER BY FINDING_DATE DESC) AS DATE_RANK
            FROM
              FINDINGS,
              LATERAL FLATTEN(INPUT => AT_RISK_ENTITIES) AS F
            QUALIFY DATE_RANK = 1
           AND SEVERITY = 'CRITICAL'
    )
  )
  THEN 
  INSERT INTO alert_event_log
    (event_timestamp,event_database, event_type,event_message)  
WITH CRITICAL_FINDINGS AS (
 SELECT ACCOUNT_DATABASE,
                SEVERITY, SCANNER_ID,
                SCANNER_SHORT_DESCRIPTION, 
                CREATED_ON::DATE AS FINDING_DATE, 
                F.VALUE:entity_object_type::VARCHAR AS ENTITY_OBJECT_TYPE,
                F.VALUE:entity_id::VARCHAR AS ENTITY_ID,
                F.VALUE:entity_name::VARCHAR AS ENTITY_NAME,
                IMPACT, 
                SUGGESTED_ACTION,
                AT_RISK_ENTITIES,
                RANK() OVER (PARTITION BY ACCOUNT_DATABASE,SCANNER_ID ORDER BY FINDING_DATE DESC) AS DATE_RANK
            FROM
              FINDINGS,
              LATERAL FLATTEN(INPUT => AT_RISK_ENTITIES) AS F
            QUALIFY DATE_RANK = 1
           AND SEVERITY = 'CRITICAL'
)
  SELECT 
    SNOWFLAKE.ALERT.SCHEDULED_TIME(), ACCOUNT_DATABASE,
    'TC_CRITICAL_FINDINGS_ALERT',
    ACCOUNT_DATABASE || ': Critical finding on '|| ENTITY_OBJECT_TYPE ||': ' || ENTITY_NAME || ': '||SCANNER_SHORT_DESCRIPTION,
FROM
  CRITICAL_FINDINGS;

--alert FOR ELEVATED PRIVILEGES
CREATE OR REPLACE ALERT ELEVATED_PRIVILEGES_ALERT
  WAREHOUSE = $WH_NAME
  SCHEDULE = '1440 MINUTE'   
  IF (EXISTS (SELECT 1
        FROM ORG_MONITORING_UTILITIES.CENTRALIZED_DATA.ELEVATED_ROLE_USERS
        WHERE  CREATED_ON > 
                COALESCE(DATEADD(DD,-1,(SNOWFLAKE.ALERT.LAST_SUCCESSFUL_SCHEDULED_TIME())),
            '2025/01/01'::TIMESTAMP_NTZ)
  ))
  THEN 
        INSERT INTO alert_event_log
            (event_timestamp,event_database, event_type,event_message)  
        SELECT SNOWFLAKE.ALERT.SCHEDULED_TIME(), ACCOUNT_DATABASE,'ELEVATED_PRIVILEGES_ALERT',ACCOUNT_DATABASE||': '|| user_name  ||' was granted the '||
        case when elevated_role is null then 'elevated privilege ' else 'elevated role ' end||
        coalesce(elevated_role, elevated_privilege)||' via this path: '||grant_path||'.'
        FROM ORG_MONITORING_UTILITIES.CENTRALIZED_DATA.ELEVATED_ROLE_USERS
        WHERE  CREATED_ON > 
                COALESCE(DATEADD(DD,-1,(SNOWFLAKE.ALERT.LAST_SUCCESSFUL_SCHEDULED_TIME())),
            '2025/01/01'::TIMESTAMP_NTZ)
;


--alert FOR ACCOUNT PARAMETERS
CREATE OR REPLACE ALERT ACCOUNT_PARAMETERS_ALERT
  WAREHOUSE = $WH_NAME
  SCHEDULE = '1440 MINUTE'   
  IF (EXISTS (SELECT 1
        FROM  ORG_MONITORING_UTILITIES.CENTRALIZED_DATA.ACCOUNT_PARAMETERS
        WHERE KEY IN 
        ('REQUIRE_STORAGE_INTEGRATION_FOR_STAGE_CREATION',-- True
        'REQUIRE_STORAGE_INTEGRATION_FOR_STAGE_OPERATION', -- True
        'PREVENT_UNLOAD_TO_INLINE_URL')
        AND VALUE ilike 'false'
        AND  COLLECTION_DATE > 
               COALESCE(DATEADD(DD,-1,(SNOWFLAKE.ALERT.LAST_SUCCESSFUL_SCHEDULED_TIME())),
               '2025/01/01'::TIMESTAMP_NTZ)
        UNION ALL
        SELECT 1
        FROM  ORG_MONITORING_UTILITIES.CENTRALIZED_DATA.ACCOUNT_PARAMETERS
        WHERE KEY IN 
        ('PRE_SIGNED_URL')
        AND VALUE ilike 'true'
        AND  COLLECTION_DATE > 
               COALESCE(DATEADD(DD,-1,(SNOWFLAKE.ALERT.LAST_SUCCESSFUL_SCHEDULED_TIME())),
               '2025/01/01'::TIMESTAMP_NTZ)
          ))
  THEN 
        INSERT INTO alert_event_log
            (event_timestamp,event_database, event_type,event_message)  
        SELECT 
            SNOWFLAKE.ALERT.SCHEDULED_TIME(), 
            ACCOUNT_DATABASE, 'ACCOUNT_PARAMETERS_ALERT',ACCOUNT_DATABASE||' account parameter '|| KEY ||' set to ' || value ||'.'
        FROM  ORG_MONITORING_UTILITIES.CENTRALIZED_DATA.ACCOUNT_PARAMETERS
        WHERE KEY IN 
        ('REQUIRE_STORAGE_INTEGRATION_FOR_STAGE_CREATION',-- True
        'REQUIRE_STORAGE_INTEGRATION_FOR_STAGE_OPERATION', -- True
        'PREVENT_UNLOAD_TO_INLINE_URL')
        AND VALUE ilike 'false'
        AND  COLLECTION_DATE > 
               COALESCE(DATEADD(DD,-1,(SNOWFLAKE.ALERT.LAST_SUCCESSFUL_SCHEDULED_TIME())),
               '2025/01/01'::TIMESTAMP_NTZ)
        UNION ALL
        SELECT
            SNOWFLAKE.ALERT.SCHEDULED_TIME(), 
            ACCOUNT_DATABASE,  'ACCOUNT_PARAMETERS_ALERT', ACCOUNT_DATABASE||' account parameter '|| KEY ||' set to ' || value ||'.'
        FROM  ORG_MONITORING_UTILITIES.CENTRALIZED_DATA.ACCOUNT_PARAMETERS
        WHERE KEY IN 
        ('PRE_SIGNED_URL')
        AND VALUE ilike 'true'
        AND  COLLECTION_DATE > 
               COALESCE(DATEADD(DD,-1,(SNOWFLAKE.ALERT.LAST_SUCCESSFUL_SCHEDULED_TIME())),
               '2025/01/01'::TIMESTAMP_NTZ)
;



## Create Alerts the Send Emails of logged events

- alert_email_events_hourly
- alert_email_events_daily

In [ ]:
--create alert to send emails hourly
CREATE OR REPLACE ALERT alert_email_events_hourly
  WAREHOUSE = $WH_NAME
  SCHEDULE = '60 minute'  
  IF (EXISTS ( 
        SELECT *
        FROM alert_event_log        
        WHERE event_timestamp BETWEEN IFNULL(SNOWFLAKE.ALERT.LAST_SUCCESSFUL_SCHEDULED_TIME(),'1970-01-01'::TIMESTAMP_NTZ)
        AND SNOWFLAKE.ALERT.SCHEDULED_TIME()
      )
  )
  THEN 
    
    DECLARE  
      table_cursor CURSOR FOR
           WITH cte_event_recipients
              (event_type,
               notification_integration,
               recipients_email) 
            AS
            (
                SELECT
                    event_type,
                    notification_integration,
                    LISTAGG(recipients_email,',')
                FROM event_recipients 
                where frequency ilike 'hourly'
                GROUP BY event_type,notification_integration
            )        
            ,cte_events(event_type,event_message, event_id) AS 
            (
                SELECT                        
                    event_type,
                     event_message,
                     array_agg(event_id)
                FROM alert_event_log
                WHERE event_timestamp>IFNULL(SNOWFLAKE.ALERT.LAST_SUCCESSFUL_SCHEDULED_TIME(),'1970-01-01'::TIMESTAMP_NTZ)
                group by 1,2
                ORDER BY EVENT_MESSAGE
            )
            SELECT 
                er.event_type email_subject,
                er.notification_integration,
                er.recipients_email,
                max(UUID_STRING()) as alert_email_id,
                LISTAGG(e.event_message||'\n') email_body,
                array_agg(event_id) email_events
            FROM cte_events e
            INNER JOIN cte_event_recipients er 
            ON e.event_type=er.event_type
            GROUP BY 1,2,3
            ;
        email_subject varchar ;
        notification_integration varchar ;
        recipients_email varchar;
        alert_email_id varchar; 
        email_body varchar;
        email_events array;
        res resultset DEFAULT (SELECT LAST_QUERY_ID());
        c1 CURSOR FOR res;
        batch_timestamp DEFAULT current_timestamp();
        
    BEGIN  -- loop through tables and extract semantic categories 
        
     for var in table_cursor DO
     begin 
        email_subject := var.email_subject ;
        notification_integration := var.notification_integration ;
        recipients_email := var.recipients_email;
        email_events := var.email_events;
        alert_email_id := var.alert_email_id; 
        email_body := var.email_body;
        let sql_stmt := 'CALL SYSTEM$SEND_EMAIL(''' ||var.notification_integration || ''','''||var.recipients_email|| ''',''' || var.email_subject|| ''',''' ||var.email_body|| ''')';
        
        EXECUTE IMMEDIATE sql_stmt; 
        
         INSERT INTO ALERT_EMAIL_LOG (
            alert_email_id,
        	email_subject,
        	notification_integration,
        	recipients_email,
        	email_events,
            query_id
            )
        select :alert_email_id, :email_subject, :notification_integration, :recipients_email, :email_events,  LAST_QUERY_ID();      
           
        EXCEPTION  -- log errors 
          WHEN statement_error THEN
          LET errm := SQLERRM;
          LET errcode := SQLCODE;
            
             INSERT INTO ALERT_EMAIL_LOG (
                alert_email_id,
                email_subject,
                notification_integration,
                recipients_email,
                email_events,
                query_id,
                error_code,
                error_message
                )
                select :alert_email_id, :email_subject, :notification_integration, :recipients_email, :email_events, LAST_QUERY_ID(), :errcode, :errm;   
        end;
     END FOR;
                                  
    END;;;


In [ ]:

--create alert to send emails daily
CREATE OR REPLACE ALERT alert_email_events_daily
  WAREHOUSE = $WH_NAME
  SCHEDULE = '1440 minute'  
  IF (EXISTS ( 
        SELECT *
        FROM alert_event_log        
        WHERE event_timestamp BETWEEN IFNULL(SNOWFLAKE.ALERT.LAST_SUCCESSFUL_SCHEDULED_TIME(),'1970-01-01'::TIMESTAMP_NTZ)
        AND SNOWFLAKE.ALERT.SCHEDULED_TIME()
      )
  )
  THEN 
    
    DECLARE  
      table_cursor CURSOR FOR
           WITH cte_event_recipients
              (event_type,
               notification_integration,
               recipients_email) 
            AS
            (
                SELECT
                    event_type,
                    notification_integration,
                    LISTAGG(recipients_email,',')
                FROM event_recipients 
                where frequency ilike 'daily'
                GROUP BY event_type,notification_integration
            )        
            ,cte_events(event_type,event_message, event_id) AS 
            (
                SELECT                        
                    event_type,
                     event_message,
                     array_agg(event_id)
                FROM alert_event_log
                WHERE event_timestamp>IFNULL(SNOWFLAKE.ALERT.LAST_SUCCESSFUL_SCHEDULED_TIME(),'1970-01-01'::TIMESTAMP_NTZ)
                group by 1,2
                ORDER BY EVENT_MESSAGE
            )
            SELECT 
                er.event_type email_subject,
                er.notification_integration,
                er.recipients_email,
                max(UUID_STRING()) as alert_email_id,
                LISTAGG(e.event_message||'\n') email_body,
                array_agg(event_id) email_events
            FROM cte_events e
            INNER JOIN cte_event_recipients er 
            ON e.event_type=er.event_type
            GROUP BY 1,2,3
            ;
        email_subject varchar ;
        notification_integration varchar ;
        recipients_email varchar;
        alert_email_id varchar; 
        email_body varchar;
        email_events array;
        res resultset DEFAULT (SELECT LAST_QUERY_ID());
        c1 CURSOR FOR res;
        batch_timestamp DEFAULT current_timestamp();
        
    BEGIN  -- loop through tables and extract semantic categories 
        
     for var in table_cursor DO
     begin 
        email_subject := var.email_subject ;
        notification_integration := var.notification_integration ;
        recipients_email := var.recipients_email;
        email_events := var.email_events;
        alert_email_id := var.alert_email_id; 
        email_body := var.email_body;
        let sql_stmt := 'CALL SYSTEM$SEND_EMAIL(''' ||var.notification_integration || ''','''||var.recipients_email|| ''',''' || var.email_subject|| ''',''' ||var.email_body|| ''')';
        
        EXECUTE IMMEDIATE sql_stmt; 
        
         INSERT INTO ALERT_EMAIL_LOG (
            alert_email_id,
        	email_subject,
        	notification_integration,
        	recipients_email,
        	email_events,
            query_id
            )
        select :alert_email_id, :email_subject, :notification_integration, :recipients_email, :email_events,  LAST_QUERY_ID();      
           
        EXCEPTION  -- log errors 
          WHEN statement_error THEN
          LET errm := SQLERRM;
          LET errcode := SQLCODE;
            
             INSERT INTO ALERT_EMAIL_LOG (
                alert_email_id,
                email_subject,
                notification_integration,
                recipients_email,
                email_events,
                query_id,
                error_code,
                error_message
                )
                select :alert_email_id, :email_subject, :notification_integration, :recipients_email, :email_events, LAST_QUERY_ID(), :errcode, :errm;   
        end;
     END FOR;
                                  
    END;
    

## Create Alert for Failed Alerts

In [ ]:
--Create alert for alert failures that have occured since the last successful execution of this alert
--https://docs.snowflake.com/en/user-guide/alerts?utm_source=legacy&utm_medium=serp&utm_term=SNOWFLAKE.ALERT.LAST_SUCCESSFUL_SCHEDULED_TIME%28%29
CREATE OR REPLACE ALERT alert_failures
  WAREHOUSE = $WH_NAME
  SCHEDULE = '1440 MINUTE'
  IF (EXISTS (  --Condition
         SELECT 1
            FROM 
                TABLE(INFORMATION_SCHEMA.ALERT_HISTORY(RESULT_LIMIT => 10000) )
                WHERE state IN ('CONDITION_FAILED','ACTION_FAILED','FAILED')
                AND SCHEDULED_TIME > SNOWFLAKE.ALERT.LAST_SUCCESSFUL_SCHEDULED_TIME()  --since this alert was last sucessfully executed
            LIMIT 1
    )
  )
  THEN --action - email notification
      CALL SYSTEM$SEND_EMAIL(
            '<integration_name>',  --notification integrations
            '<email.address1@mass.gov>,<email.address2@mass.gov>',  --recipient list (comma separated for multiple recipients)
            'Alert Failures',  --email subject
            (WITH CTE AS
                (SELECT 'ALERT: ' || ifnull(NAME,'') ||
                    ' - DB: ' || ifnull(DATABASE_NAME,'') || ' - SCH: ' || ifnull(SCHEMA_NAME,'') || 
                    ' - STATE: ' || ifnull(STATE,'') || ' - ERROR: ' || SQL_ERROR_MESSAGE  as mail_text
                    FROM 
                    TABLE(INFORMATION_SCHEMA.ALERT_HISTORY(RESULT_LIMIT => 10000)) 
                    WHERE state IN ('CONDITION_FAILED','ACTION_FAILED','FAILED')
                    AND SCHEDULED_TIME > SNOWFLAKE.ALERT.LAST_SUCCESSFUL_SCHEDULED_TIME() --since this alert was last sucessfully executed
                )
                SELECT listagg(mail_text||'\n') FROM CTE
            ));


## Popultate Event Recipients Table

## Update email recipient list in lines 3-5


In [ ]:
--example
INSERT INTO event_recipients VALUES 
('TC_CRITICAL_FINDINGS_ALERT','GENERAL_EMAIL_INT','<email_address1, email_address2, email_address ...>', 'daily'),
('ELEVATED_PRIVILEGES_ALERT','GENERAL_EMAIL_INT','<email_address1, email_address2, email_address ...>', 'daily');
('ACCOUNT_PARAMETERS','GENERAL_EMAIL_INT','<email_address1, email_address2, email_address ...>', 'daily');

## Verify Alert Creation & Resume Alerts

In [ ]:
SHOW ALERTS IN SCHEMA;

In [ ]:
ALTER ALERT ACCOUNT_PARAMETERS_ALERT RESUME;
ALTER ALERT ALERT_EMAIL_EVENTS_DAILY RESUME;
ALTER ALERT ALERT_EMAIL_EVENTS_HOURLY RESUME;
ALTER ALERT ALERT_FAILURES RESUME;
ALTER ALERT ELEVATED_PRIVILEGES_ALERT RESUME;
ALTER ALERT TC_CRITICAL_FINDINGS_ALERT RESUME;


In [ ]:
-- ALTER ALERT ACCOUNT_PARAMETERS_ALERT SUSPEND;
-- ALTER ALERT ALERT_EMAIL_EVENTS_DAILY SUSPEND;
-- ALTER ALERT ALERT_EMAIL_EVENTS_HOURLY SUSPEND;
-- ALTER ALERT ALERT_FAILURES SUSPEND;
-- ALTER ALERT ELEVATED_PRIVILEGES_ALERT SUSPEND;
-- ALTER ALERT TC_CRITICAL_FINDINGS_ALERT SUSPEND;


# Sample Report Queries

In [ ]:
-----REPORT QUERIES

--FINDINGS SEVERITY SUMMARY
WITH RECENT_FINDINGS AS (select
        ACCOUNT_DATABASE,
            SCANNER_PACKAGE_NAME,
            CREATED_ON::DATE AS FINDING_DATE, 
            SEVERITY, 
            SCANNER_ID,
            RANK() OVER (PARTITION BY SCANNER_ID ORDER BY FINDING_DATE DESC) AS DATE_RANK, TOTAL_AT_RISK_COUNT, EVENT_ID, AT_RISK_ENTITIES
        FROM FINDINGS
        QUALIFY DATE_RANK = 1 AND TOTAL_AT_RISK_COUNT >0
        ORDER BY SCANNER_ID, EVENT_ID)
    SELECT 
        ACCOUNT_DATABASE,
        SCANNER_PACKAGE_NAME,
        FINDING_DATE,
        SEVERITY,  
        COUNT(DISTINCT SCANNER_ID) AS SEVERITY_COUNT
    FROM RECENT_FINDINGS
    GROUP BY ALL
    ORDER BY SCANNER_PACKAGE_NAME,
        CASE WHEN SEVERITY = 'CRITICAL' THEN 1
                    WHEN SEVERITY = 'HIGH' THEN 2
                    WHEN SEVERITY = 'MEDIUM' THEN 3
                    WHEN SEVERITY = 'LOW' THEN 4 END;
                    
--LATEST  FINDINGS DETAILS
SELECT ACCOUNT_DATABASE,
        SEVERITY, SCANNER_ID,
        SCANNER_SHORT_DESCRIPTION, 
        CREATED_ON::DATE AS FINDING_DATE, 
        F.VALUE:entity_object_type::VARCHAR AS ENTITY_OBJECT_TYPE,
        F.VALUE:entity_id::VARCHAR AS ENTITY_ID,
        F.VALUE:entity_name::VARCHAR AS ENTITY_NAME,
        IMPACT, 
        SUGGESTED_ACTION,
        AT_RISK_ENTITIES,
        RANK() OVER (PARTITION BY ACCOUNT_DATABASE,SCANNER_ID ORDER BY FINDING_DATE DESC) AS DATE_RANK
    FROM
      FINDINGS,
      LATERAL FLATTEN(INPUT => AT_RISK_ENTITIES) AS F
    QUALIFY DATE_RANK = 1
    ORDER BY SCANNER_PACKAGE_NAME,
        CASE WHEN SEVERITY = 'CRITICAL' THEN 1
                    WHEN SEVERITY = 'HIGH' THEN 2
                    WHEN SEVERITY = 'MEDIUM' THEN 3
                    WHEN SEVERITY = 'LOW' THEN 4 END;


--RISK COUNT BY FINDING
WITH FINDINGS AS (
        SELECT CURRENT_ACCOUNT() AS ACCOUNT, 
            CURRENT_ACCOUNT_NAME() AS ACCOUNT_NAME,
            SCANNER_PACKAGE_NAME,
            CREATED_ON::DATE AS FINDING_DATE,
            SCANNER_SHORT_DESCRIPTION,
            SEVERITY, 
            TOTAL_AT_RISK_COUNT,
            RANK() OVER (PARTITION BY SCANNER_ID ORDER BY FINDING_DATE DESC) AS DATE_RANK, 
        FROM SNOWFLAKE.TRUST_CENTER.FINDINGS
        QUALIFY DATE_RANK = 1 AND TOTAL_AT_RISK_COUNT >0)
    SELECT ACCOUNT, 
        ACCOUNT_NAME,
        SCANNER_PACKAGE_NAME,
        FINDING_DATE,
        SCANNER_SHORT_DESCRIPTION,
        SEVERITY,   
        SUM(TOTAL_AT_RISK_COUNT) AS TOTAL_AT_RISK_COUNT
    FROM FINDINGS
    GROUP BY ALL
    ORDER BY SCANNER_PACKAGE_NAME,
        CASE WHEN SEVERITY = 'CRITICAL' THEN 1
                    WHEN SEVERITY = 'HIGH' THEN 2
                    WHEN SEVERITY = 'MEDIUM' THEN 3
                    WHEN SEVERITY = 'LOW' THEN 4 END;

--USERS WITH ELEVATED ROLES AND PRIVILEGES
SELECT * FROM ORG_MONITORING_UTILITIES.CENTRALIZED_DATA.ELEVATED_ROLE_USERS; 

--ACCOUNT PARAMETER VALUES
SELECT * FROM  ORG_MONITORING_UTILITIES.CENTRALIZED_DATA.ACCOUNT_PARAMETERS
WHERE KEY IN 
('REQUIRE_STORAGE_INTEGRATION_FOR_STAGE_CREATION',
'REQUIRE_STORAGE_INTEGRATION_FOR_STAGE_OPERATION', 
'PREVENT_UNLOAD_TO_INLINE_URL','PRE_SIGNED_URL');

